# Washington_RPV_2020_Precincts_05_08_2023

## Background:
- We received a request for voter registration by party and voter turnout by primary contest (Republican and Democratic) for the 2022 Primary Election, voter turnout for the 2022 General Election, 2020 VAP by race/ethnicity, 2021 CVAP by race/ethnicity, 2020 Presidential and Gubernatorial contests, and 2022 Senate results on 2020 Precincts.

## Approach:
Voter file data was provided by L2. Voter file data is retrieved at the individual level and aggregated to 2020 precincts using a spatial join. Not all individuals are able to be aggregated as they do not all contain a latitude, longitude coordinate (i.e. not geocoded). There are 32 instances where the lat/long of a voter falls in potentially two precincts, to select the correct precinct we matched voter file-assigned precincts and counties to the 2020 precinct shapefile. Party registration data is modeled by L2 based entirely on academic modeling. Because voters can vote for candidates in any party, the primary voter turnout by party is based on the modeled registration (only 4 individuals appear to be an exception to this). Voter file data on 2020 precincts is then grouped by precinct assignment and joined with the precinct file containing 2020 election results.

2020 election results and precinct boundaries are provided by the [Voting Election and Science Team](https://dataverse.harvard.edu/file.xhtml?fileId=5007851&version=40.0).

2022 Senate election results were retrieved from the Washington Secretary of State [here](https://www.sos.wa.gov/elections/research/2022-general-election.aspx) and joined to the 2022 shapefile also retrieved from the SOS [here](https://www.sos.wa.gov/elections/research/Precinct-Shapefiles.aspx). These data were disaggregated using a modified Voting Age Population (VAP) variable where the adult incarcerated population is removed to calculate data at the 2020 block level. The results differ by a total of 4 for Murray and match the official totals exactly for other candidates.

Other 2020 block level data include CVAP and VAP by race. The 2020 VAP data by race is readily provided by the Census in the PL 94-171 dataset. We used the "Alone" values where indicated above in the Fields section (a single variable in the PL data) and summed all respective columns including that race/ethnicity for those that are listed as "any part". 2021 CVAP data is provided by the Census at the block group level and the RDH has disaggregated this data to the block level using a racially-sensitive disaggregation to maintain racial distribution inside the block group. 

All 2020 block level data (2022 Senate, 2020 VAP, 2021 CVAP) was joined together on the GEOID20 field and assigned a "best fit" precinct using [maup](https://github.com/mggg/maup). These data were then grouped and summed by this assignment to then have all data (2020 election results, 2020 voter file, 2022 Senate results, 2020 VAP, and 2021 CVAP) on 2020 precinct shapes. Due to rounding differences, there can be up to a maximum of 4 (votes or people) fewer in the final file than in the inputs for a given column, for the entire state.

The data was checked for accuracy and exported as a CSV.

Please note that the 2021 CVAP data for CVAP_ASN21 and CVAP_AIA21 represent alone or in combination (Asian Alone + Asian and White, and Native American Alone + Native American and Black + Native American and White), which is slightly different than the VAP values, as those are represented as just "Alone". Moreover, the CVAP_BLK21 represents a sum of all available CVAP data (Black Alone + Black and White + Black and Native American) but the VAP data is more encompassing, including more fields as the PL data provides more racial categories. It is also to be noted that all CVAP data is Non-Hispanic, while in VAP it varies depending on the field (as noted in the description and column name).

For a 'raw-from-source' folder containing CVAP and PL data and 2020 and 2022 election results, contact info@redistrictingdatahub.org. Individual level voter file data cannot be provided.

Import necessary libraries

In [1]:
import os
import pandas as pd
import geopandas as gp
import maup
pd.set_option('display.max_columns', None)
wd = os.path.join(os.getcwd(),'raw-from-source')

Import all necessary datasets (voterfile at individual-level, CVAP at block-level, PL data at block-level, Senate 2022 results at block-level, and 2020 election results at 2020 precinct-level)

In [2]:
vf = pd.read_csv(os.path.join(wd,'wa_voter_data.csv'))
cvap = gp.read_file(os.path.join(os.path.join(wd,'wa_cvap_2021_2020_b'),'wa_cvap_2021_2020_b.shp'))
pl = pd.read_csv(os.path.join(os.path.join(wd,'wa_pl2020_b'),'wa_pl2020_b.csv'))
sen22 = gp.read_file(os.path.join(os.path.join(wd,'wa_2022_gen_2020_blocks'),'wa_2022_gen_2020_blocks.shp'))

prec = gp.read_file(os.path.join(os.path.join(wd,'wa_2020_clean'),'wa_2020_clean.shp'))
#Subset necessary columns for precinct 
prec = prec[['COUNTY', 'PRECCODE', 'COUNTYNAME', 'ST_CODE', 'PRECNAME', 'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PRESLAR', 'G20PRESKEN', 'G20PREOWRI', 'G20GOVDINS', 'G20GOVRCUL', 'G20GOVOWRI', 'geometry']]

C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_5752\3103163987.py:3: DtypeWarning: Columns (29,31,35,37,38,55) have mixed types. Specify dtype option on import or set low_memory=False.
  pl = pd.read_csv(os.path.join(os.path.join(wd,'wa_pl2020_b'),'wa_pl2020_b.csv'))


Make voterfile into point shapefile using lat/long coordinates and join spatial join to the 2020 precinct shapefile

In [3]:
vf['lat'] = vf['lat'].astype(float)
vf['lon'] = vf['lon'].astype(float)
points = gp.GeoDataFrame(vf,geometry = gp.points_from_xy(vf.lon, vf.lat, crs='EPSG:4326'))
points = points.to_crs(prec.crs)
joined = points.sjoin(prec,how='left')

Clean duplicate voters by giving them one assignment

In [4]:
joined['dup']= joined.duplicated(subset='lalvoterid')
dups = joined[joined['dup']==True]
dups_ids = list(dups['lalvoterid'].unique())
joined_sub_dups = joined[joined['lalvoterid'].isin(dups_ids)]
no_dups = joined[~joined['lalvoterid'].isin(dups_ids)]
dups_dict = {'LALWA118506730':'CH00000490','LALWA582670':'GY00000068','LALWA139148': 'CH00000540', 'LALWA521555976': 'CH00000183', 'LALWA2061576': 'KT00000007', 'LALWA2087785': 'PI00027471', 'LALWA562843102': 'PI00031856', 'LALWA491602': 'FE00000111', 'LALWA2643741': 'WM00000182', 'LALWA608269735': 'PI00027471', 'LALWA2287638': 'PI00002158', 'LALWA623473528': 'FE00000105', 'LALWA581915918': 'WM00000182', 'LALWA2936': 'AD00000516', 'LALWA2643798': 'WM00000182', 'LALWA146220640': 'YA00001708', 'LALWA2215295': 'PI00027477', 'LALWA2940': 'AD00000516', 'LALWA2374613': 'PI00027477', 'LALWA603195119': 'MA00000232', 'LALWA2364259': 'PI00027477', 'LALWA554089': 'MA00000232', 'LALWA2055990': 'KT00000007', 'LALWA2935': 'AD00000516', 'LALWA562844032': 'PI00027477', 'LALWA5855': 'AD00000516', 'LALWA5854': 'AD00000516', 'LALWA462808320': 'WM00000182', 'LALWA608341587': 'MA00000232', 'LALWA488828': 'FE00000105', 'LALWA3152607': 'AD00000516', 'LALWA2369239': 'PI00027477'}
rows = []
for i in dups_ids:
    sub = joined_sub_dups[joined_sub_dups['lalvoterid']==i]
    prec_to_match = dups_dict.get(i)
    row = sub[sub['ST_CODE']==prec_to_match]
    if len(row)!=1:
        print(i)
        display(row)
        display(sub)
    rows.append(row)
to_concat_back = pd.concat(rows)
full = pd.concat([to_concat_back,no_dups])
print('Number of voters who had duplicate assignments: ', str(len(dups_dict.keys())))
if len(full)==len(points):
    print('All voters have one spatial assignment.')

Number of voters who had duplicate assignments:  32
All voters have one spatial assignment.


Create pivot table for ballot type selected in the primary (REP or DEM) to aggregate data to 2020 precincts

In [5]:
full = full[['lalvoterid','parties_description','ball_type_prim','gen','ST_CODE']]
full['ball_type_prim'] = full['ball_type_prim'].fillna('No Primary Vote')
full['gen'] = full['gen'].fillna('No General Vote')
full['counts'] = 1
ball_type = full.pivot_table(index='ST_CODE',columns = ['ball_type_prim'],values=['counts'],aggfunc=sum)
ball_type.reset_index(drop=False,inplace=True)
ball_type.columns = ball_type.columns.droplevel(0)
ball_type.rename(columns = {ball_type.columns[0]:"ST_CODE",'D':'DEM_PRIM_VOTE22','R':'REP_PRIM_VOTE22','O':'OTH_PRIM_VOTE22'}, inplace = True)
ball_type.drop(columns = 'No Primary Vote',inplace=True)
display(ball_type.head())

ball_type_prim,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22
0,AD00000111,24.0,16.0,59.0
1,AD00000112,27.0,8.0,49.0
2,AD00000113,34.0,6.0,83.0
3,AD00000114,33.0,8.0,86.0
4,AD00000115,40.0,19.0,104.0


Create pivot table for modeled party registration to aggregate data to 2020 precincts

In [6]:
parties = full.pivot_table(index='ST_CODE',columns = ['parties_description'],values=['counts'],aggfunc=sum)
parties.reset_index(drop=False,inplace=True)
parties.columns = parties.columns.droplevel(0)
parties.rename(columns = {parties.columns[0]:"ST_CODE",'Democratic':'DEM_PRIM_REG22','Republican':'REP_PRIM_REG22','Non-Partisan':'NON_PRIM_REG22'}, inplace = True)
parties.head()

parties_description,ST_CODE,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22
0,AD00000111,58.0,78.0,86.0
1,AD00000112,59.0,46.0,89.0
2,AD00000113,73.0,50.0,131.0
3,AD00000114,69.0,43.0,124.0
4,AD00000115,65.0,43.0,155.0


Create pivot table for general election voter turnout to aggregate data to 2020 precincts

In [7]:
gen = full.pivot_table(index='ST_CODE',columns = ['gen'],values=['counts'],aggfunc=sum)
gen.reset_index(drop=False,inplace=True)
gen.columns = gen.columns.droplevel(0)
gen.rename(columns = {gen.columns[0]:"ST_CODE",'Y':'VOTED_GEN22'},inplace=True)
gen.drop(columns = 'No General Vote',inplace=True)
gen.head()

gen,ST_CODE,VOTED_GEN22
0,AD00000111,138.0
1,AD00000112,115.0
2,AD00000113,182.0
3,AD00000114,153.0
4,AD00000115,183.0


Join all pivot tables together 

In [8]:
joined_ball_type_parties = pd.merge(ball_type,parties,on='ST_CODE',how='outer',indicator=True)
print('Ballot type and registration join successful? ', str(len(joined_ball_type_parties[joined_ball_type_parties['_merge']!='both'])==0))
joined_ball_type_parties.drop(columns = '_merge',inplace=True)
joined_ball_type_parties.head()

Ballot type and registration join successful?  True


,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22
0,AD00000111,24.0,16.0,59.0,58.0,78.0,86.0
1,AD00000112,27.0,8.0,49.0,59.0,46.0,89.0
2,AD00000113,34.0,6.0,83.0,73.0,50.0,131.0
3,AD00000114,33.0,8.0,86.0,69.0,43.0,124.0
4,AD00000115,40.0,19.0,104.0,65.0,43.0,155.0


In [9]:
joined_all_vf= pd.merge(joined_ball_type_parties,gen,on='ST_CODE',how='outer',indicator=True)
print('All voterfile at precinct level join successful? ', str(len(joined_all_vf[joined_all_vf['_merge']!='both'])==0))
joined_all_vf.drop(columns = '_merge',inplace=True)
joined_all_vf.head()

All voterfile at precinct level join successful?  True


,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22,VOTED_GEN22
0,AD00000111,24.0,16.0,59.0,58.0,78.0,86.0,138.0
1,AD00000112,27.0,8.0,49.0,59.0,46.0,89.0,115.0
2,AD00000113,34.0,6.0,83.0,73.0,50.0,131.0,182.0
3,AD00000114,33.0,8.0,86.0,69.0,43.0,124.0,153.0
4,AD00000115,40.0,19.0,104.0,65.0,43.0,155.0,183.0


Join all pivoted voter file data with election data on 2020 precincts

In [10]:
joined_all_precinct = pd.merge(joined_all_vf,prec,on='ST_CODE',how='outer',indicator=False)
vf_cols = list(joined_all_vf.columns)
for i in vf_cols:
    joined_all_precinct[i].fillna(0.0)
joined_all_precinct.head()

,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22,VOTED_GEN22,COUNTY,PRECCODE,COUNTYNAME,PRECNAME,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREGHAW,G20PRESLAR,G20PRESKEN,G20PREOWRI,G20GOVDINS,G20GOVRCUL,G20GOVOWRI,geometry
0,AD00000111,24.0,16.0,59.0,58.0,78.0,86.0,138.0,53001,111,Adams,RITZVILLE WARD #1,48,118,3,1,0,0,1,42,129,2,"POLYGON ((-13177013.223 5964405.679, -13177014..."
1,AD00000112,27.0,8.0,49.0,59.0,46.0,89.0,115.0,53001,112,Adams,RITZVILLE WARD #2,34,100,1,0,0,0,1,31,102,0,"POLYGON ((-13178829.249 5961367.230, -13178837..."
2,AD00000113,34.0,6.0,83.0,73.0,50.0,131.0,182.0,53001,113,Adams,RITZVILLE WARD #3,66,147,2,3,0,1,0,54,168,1,"POLYGON ((-13176417.014 5962897.730, -13176412..."
3,AD00000114,33.0,8.0,86.0,69.0,43.0,124.0,153.0,53001,114,Adams,RITZVILLE WARD #4,52,147,2,0,0,0,1,46,157,0,"POLYGON ((-13177483.251 5962666.882, -13177500..."
4,AD00000115,40.0,19.0,104.0,65.0,43.0,155.0,183.0,53001,115,Adams,RITZVILLE WARD #5,50,161,7,0,0,0,0,43,174,0,"POLYGON ((-13176679.523 5960452.928, -13176773..."


Check number of voters who are omitted in the aggregated data due to no lat/long

In [11]:
print('Original number of registered voters in L2 file: ', str(len(vf)))
all_reg_agg = joined_all_precinct['DEM_PRIM_REG22'].sum()+ joined_all_precinct['REP_PRIM_REG22'].sum()+ joined_all_precinct['NON_PRIM_REG22'].sum()
print('Number of registered voters that get aggregated to precincts: ', str(all_reg_agg))
print('Number of registered voters in L2 file that did not get aggregated to precincts: ', str(len(vf)-all_reg_agg))

Original number of registered voters in L2 file:  4880684
Number of registered voters that get aggregated to precincts:  4866439.0
Number of registered voters in L2 file that did not get aggregated to precincts:  14245.0


In [12]:
original_rep_prim_voters = len(vf[vf['ball_type_prim']=='R'])
rep_voters_agg = joined_all_precinct['REP_PRIM_VOTE22'].sum()

In [13]:
print('Original number of Republican primary voters in L2 file: ', str(original_rep_prim_voters))
print('Number of Republican primary voters that get aggregated to precincts: ', str(rep_voters_agg))
print('Number of Republican primary voters in L2 file that did not get aggregated to precincts: ', str(original_rep_prim_voters-rep_voters_agg))

Original number of Republican primary voters in L2 file:  670547
Number of Republican primary voters that get aggregated to precincts:  668333.0
Number of Republican primary voters in L2 file that did not get aggregated to precincts:  2214.0


In [14]:
original_dem_prim_voters = len(vf[vf['ball_type_prim']=='D'])
dem_voters_agg = joined_all_precinct['DEM_PRIM_VOTE22'].sum()

In [15]:
print('Original number of Democractic primary voters in L2 file: ', str(original_dem_prim_voters))
print('Number of Democratic primary voters that get aggregated to precincts: ', str(dem_voters_agg))
print('Number of Democratic primary voters in L2 file that did not get aggregated to precincts: ', str(original_dem_prim_voters-dem_voters_agg))

Original number of Democractic primary voters in L2 file:  1080287
Number of Democratic primary voters that get aggregated to precincts:  1077306.0
Number of Democratic primary voters in L2 file that did not get aggregated to precincts:  2981.0


In [16]:
original_gen_voters = len(vf[vf['gen']=='Y'])
gen_voters_agg = joined_all_precinct['VOTED_GEN22'].sum()

In [17]:
print('Original number of general election voters in L2 file: ', str(original_gen_voters))
print('Number of general election voters that get aggregated to precincts: ', str(gen_voters_agg))
print('Number of general election voters in L2 file that did not get aggregated to precincts: ', str(original_gen_voters-gen_voters_agg))

Original number of general election voters in L2 file:  2975866
Number of general election voters that get aggregated to precincts:  2967292.0
Number of general election voters in L2 file that did not get aggregated to precincts:  8574.0


Query out necessary PL fields and sum where necessary

In [18]:
any_part_black_cols = ['P0030004', 'P0030011', 'P0030016', 'P0030017', 'P0030018', 'P0030019', 'P0030027', 'P0030028', 'P0030029', 'P0030030', 'P0030037', 'P0030038', 'P0030039', 'P0030040', 'P0030041', 'P0030042', 'P0030048', 'P0030049', 'P0030050', 'P0030051', 'P0030052', 'P0030053', 'P0030058', 'P0030059', 'P0030060', 'P0030061', 'P0030064', 'P0030065', 'P0030066', 'P0030067', 'P0030069', 'P0030071']
any_part_nhsp_black_cols = ['P0040006', 'P0040013', 'P0040018', 'P0040019', 'P0040020', 'P0040021', 'P0040029', 'P0040030', 'P0040031', 'P0040032', 'P0040039', 'P0040040', 'P0040041', 'P0040042', 'P0040043', 'P0040044', 'P0040050', 'P0040051', 'P0040052', 'P0040053', 'P0040054', 'P0040055', 'P0040060', 'P0040061', 'P0040062', 'P0040063', 'P0040066', 'P0040067', 'P0040068', 'P0040069', 'P0040071', 'P0040073']
pl_cols_to_keep = ['GEOCODE','P0010001','P0030001','P0040005','P0040007','P0040008','P0040009','P0040010','P0030008','P0040002']
to_query = pl_cols_to_keep+any_part_black_cols+any_part_nhsp_black_cols
pl['GEOCODE'] = pl['GEOCODE'].apply(lambda x: str(x).zfill(15))
pl_queried = pl[to_query]
pl_queried['ANY_PART_BLK_NHSP_VAP'] = pl_queried[any_part_nhsp_black_cols].sum(axis=1)
pl_queried['ANY_PART_BLK_VAP'] = pl_queried[any_part_black_cols].sum(axis=1)
pl_cols_to_keep = pl_cols_to_keep+['ANY_PART_BLK_NHSP_VAP','ANY_PART_BLK_VAP']
pl_queried2 = pl_queried[pl_cols_to_keep]
pl_queried2.rename(columns = {'GEOCODE':'GEOID20','P0010001':'TOTAL_POP','P0030001':'TOTAL_VAP','P0040005':'WHT_NHSP_VAP','P0040007':'AIAN_NHSP_VAP','P0040008':'ASN_NHSP_VAP','P0040009':'NHOPI_NHSP_VAP','P0040010':'OTH_NHSP_VAP','P0030008':'OTH_HISP_VAP','P0040002':'HISP_VAP'},inplace=True)
pl_queried2.head()

C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_5752\1985261131.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_queried['ANY_PART_BLK_NHSP_VAP'] = pl_queried[any_part_nhsp_black_cols].sum(axis=1)
C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_5752\1985261131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_queried['ANY_PART_BLK_VAP'] = pl_queried[any_part_black_cols].sum(axis=1)
C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_5752\1985261131.py:11: SettingWithCopyWarning: 
A value 

,GEOID20,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP
0,530019501001000,11,7,7,0,0,0,0,0,0,0,0
1,530019501001001,0,0,0,0,0,0,0,0,0,0,0
2,530019501001002,0,0,0,0,0,0,0,0,0,0,0
3,530019501001003,0,0,0,0,0,0,0,0,0,0,0
4,530019501001004,0,0,0,0,0,0,0,0,0,0,0


Query out necessary CVAP columns and join with the queried PL data

In [19]:
cvap_cols_to_keep = ['GEOID20','CVAP_TOT21','CVAP_WHT21','CVAP_BLK21','CVAP_AIA21','CVAP_ASN21','CVAP_NHP21','CVAP_2OM21','CVAP_HSP21','geometry']
cvap['GEOID20'] = cvap['GEOID20'].apply(lambda x: str(x).zfill(15))
cvap_sub = cvap[cvap_cols_to_keep]

cvap_pl = pd.merge(cvap_sub,pl_queried2,on='GEOID20',how='outer',indicator=True)
display(cvap_pl[cvap_pl['_merge']!='both'])
cvap_pl.drop(columns = '_merge',inplace=True)

,GEOID20,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,geometry,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,_merge


Query out necessary columns from 2022 elections sresuts and join to CVAP/PL data

In [20]:
sen22_cols_to_keep = ['GEOID20','G22SENDMUR','G22SENRSMI','G22SENOWRI']
sen22['GEOID20'] = sen22['GEOID20'].apply(lambda x: str(x).zfill(15))
sen22_sub = sen22[sen22_cols_to_keep]

cvap_pl_sen22 = pd.merge(cvap_pl,sen22_sub,on='GEOID20',how='outer',indicator=True)
for i in sen22_cols_to_keep:
    if i.startswith('G22'):
        cvap_pl_sen22[i] = cvap_pl_sen22[i].fillna(0.0)

Assign all blocks to 2020 precincts

In [21]:
prec.set_index('ST_CODE',inplace=True,drop=False)
cvap_pl_sen22 = cvap_pl_sen22.to_crs(prec.crs)
cvap_pl_sen22['ST_CODE'] = maup.assign(cvap_pl_sen22, prec)
cvap_pl_sen22.head()

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i


,GEOID20,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,geometry,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,G22SENDMUR,G22SENRSMI,G22SENOWRI,_merge,ST_CODE
0,530110405072005,21.31,18.45,0.00,0.09,1.82,0.0,0.0,0.83,"POLYGON ((-13617309.465 5712545.401, -13617308...",37,24,18,0,1,0,0,0,4,0,3,3.09,4.18,0.00,both,CR00000909
1,530110405121028,17.58,13.84,0.00,0.23,0.00,0.0,0.0,0.00,"POLYGON ((-13615909.061 5713367.973, -13615887...",29,21,15,1,0,0,0,0,0,2,2,6.12,7.83,0.03,both,CR00000914
2,530110423001016,21.37,20.19,0.00,0.48,0.00,0.0,0.0,0.00,"POLYGON ((-13657040.872 5722131.510, -13657040...",24,18,17,0,0,0,0,0,0,0,0,7.02,2.46,0.04,both,CR00000110
3,530110423001012,33.24,22.57,0.42,2.86,0.00,0.0,0.0,2.27,"POLYGON ((-13656913.969 5722259.194, -13656913...",29,28,19,6,0,0,0,2,2,1,1,10.92,3.82,0.06,both,CR00000110
4,530110413331019,25.35,19.48,0.00,0.00,0.00,0.0,0.0,4.04,"POLYGON ((-13639265.020 5725010.191, -13639263...",26,26,20,0,0,0,0,1,5,0,0,6.35,5.97,0.00,both,CR00000654


In [22]:
cvap_pl_sen22['ST_CODE'].fillna('NO PRECINCT')

0         CR00000909
1         CR00000914
2         CR00000110
3         CR00000110
4         CR00000654
             ...    
158088    LE00000014
158089    CH00000610
158090    LE00000049
158091    LE00000022
158092    CH00000164
Name: ST_CODE, Length: 158093, dtype: object

Aggregate block data to 2020 precincts using the assigned precinct 

In [23]:
grouped_blocks = cvap_pl_sen22.groupby('ST_CODE').sum()
grouped_blocks.reset_index(inplace=True)
grouped_blocks.head()

,ST_CODE,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,G22SENDMUR,G22SENRSMI,G22SENOWRI
0,AD00000111,269.75,229.40,0.00,2.39,11.20,6.27,0.0,17.25,401,330,261,8,6,0,0,13,37,0,0,45.04,100.97,0.00
1,AD00000112,212.69,196.55,5.56,2.17,2.80,3.36,0.0,9.32,320,238,210,0,0,0,0,4,12,1,1,30.00,87.01,0.00
2,AD00000113,306.36,272.92,16.68,2.49,4.45,0.00,0.0,10.88,343,276,245,0,4,0,0,2,10,3,3,50.99,131.01,1.99
3,AD00000114,333.00,275.17,22.23,2.78,2.67,0.00,0.0,28.26,371,300,247,4,1,0,3,15,26,4,5,37.99,117.00,1.99
4,AD00000115,290.81,276.25,5.56,1.38,0.89,0.00,0.0,4.35,332,262,248,0,1,0,0,1,4,1,1,41.97,150.01,0.99


Confirm all blocks receive a precinct assignment

In [24]:
no_assignment = grouped_blocks[grouped_blocks['ST_CODE']=='NO PRECINCT']
grouped_blocks_keep = grouped_blocks[grouped_blocks['ST_CODE']!='NO PRECINCT']
display(no_assignment)

,ST_CODE,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,G22SENDMUR,G22SENRSMI,G22SENOWRI


Join all data together (PL/CVAP/Senate dataframe with voterfile/2020 election dataframe)

In [25]:
joined_demo_votes = pd.merge(joined_all_precinct,grouped_blocks_keep,on='ST_CODE',how='outer',indicator=True)
display(joined_demo_votes[joined_demo_votes['_merge']!='both'])

,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22,VOTED_GEN22,COUNTY,PRECCODE,COUNTYNAME,PRECNAME,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREGHAW,G20PRESLAR,G20PRESKEN,G20PREOWRI,G20GOVDINS,G20GOVRCUL,G20GOVOWRI,geometry,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,G22SENDMUR,G22SENRSMI,G22SENOWRI,_merge
3892,KI00008888,NaN,NaN,NaN,26.0,58.0,NaN,1.0,53033,8888,King,ELECTIONS OFFICE,42,12,1,1,0,0,0,35,12,0,"POLYGON ((-13606545.458 6018891.704, -13606545...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7453,SN23918907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53061,23918907,Snohomish,MARYSVILLE LIFT STATION,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-13602800.586 6138182.782, -13602801...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Fill any null columns to 0

In [26]:
joined_demo_votes.drop(columns = ['_merge','geometry'],inplace=True)
for i in joined_demo_votes.columns:
    joined_demo_votes[i] = joined_demo_votes[i].fillna(0.0)
display(joined_demo_votes.head())

,ST_CODE,DEM_PRIM_VOTE22,OTH_PRIM_VOTE22,REP_PRIM_VOTE22,DEM_PRIM_REG22,NON_PRIM_REG22,REP_PRIM_REG22,VOTED_GEN22,COUNTY,PRECCODE,COUNTYNAME,PRECNAME,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREGHAW,G20PRESLAR,G20PRESKEN,G20PREOWRI,G20GOVDINS,G20GOVRCUL,G20GOVOWRI,CVAP_TOT21,CVAP_WHT21,CVAP_BLK21,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_HSP21,TOTAL_POP,TOTAL_VAP,WHT_NHSP_VAP,AIAN_NHSP_VAP,ASN_NHSP_VAP,NHOPI_NHSP_VAP,OTH_NHSP_VAP,OTH_HISP_VAP,HISP_VAP,ANY_PART_BLK_NHSP_VAP,ANY_PART_BLK_VAP,G22SENDMUR,G22SENRSMI,G22SENOWRI
0,AD00000111,24.0,16.0,59.0,58.0,78.0,86.0,138.0,53001,111,Adams,RITZVILLE WARD #1,48,118,3,1,0,0,1,42,129,2,269.75,229.40,0.00,2.39,11.20,6.27,0.0,17.25,401.0,330.0,261.0,8.0,6.0,0.0,0.0,13.0,37.0,0.0,0.0,45.04,100.97,0.00
1,AD00000112,27.0,8.0,49.0,59.0,46.0,89.0,115.0,53001,112,Adams,RITZVILLE WARD #2,34,100,1,0,0,0,1,31,102,0,212.69,196.55,5.56,2.17,2.80,3.36,0.0,9.32,320.0,238.0,210.0,0.0,0.0,0.0,0.0,4.0,12.0,1.0,1.0,30.00,87.01,0.00
2,AD00000113,34.0,6.0,83.0,73.0,50.0,131.0,182.0,53001,113,Adams,RITZVILLE WARD #3,66,147,2,3,0,1,0,54,168,1,306.36,272.92,16.68,2.49,4.45,0.00,0.0,10.88,343.0,276.0,245.0,0.0,4.0,0.0,0.0,2.0,10.0,3.0,3.0,50.99,131.01,1.99
3,AD00000114,33.0,8.0,86.0,69.0,43.0,124.0,153.0,53001,114,Adams,RITZVILLE WARD #4,52,147,2,0,0,0,1,46,157,0,333.00,275.17,22.23,2.78,2.67,0.00,0.0,28.26,371.0,300.0,247.0,4.0,1.0,0.0,3.0,15.0,26.0,4.0,5.0,37.99,117.00,1.99
4,AD00000115,40.0,19.0,104.0,65.0,43.0,155.0,183.0,53001,115,Adams,RITZVILLE WARD #5,50,161,7,0,0,0,0,43,174,0,290.81,276.25,5.56,1.38,0.89,0.00,0.0,4.35,332.0,262.0,248.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,41.97,150.01,0.99


Clean and organize columns

In [27]:
joined_demo_votes.rename(columns = {'ST_CODE':'UNIQUE_ID','COUNTY':'COUNTYFP20','PRECNAME':'PRECINCT20'},inplace=True)
joined_demo_votes_col_order = ['UNIQUE_ID','COUNTYFP20','COUNTYNAME','PRECINCT20','G20PREDBID', 'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PRESLAR', 'G20PRESKEN', 'G20PREOWRI', 'G20GOVDINS', 'G20GOVRCUL', 'G20GOVOWRI', 
                              'G22SENDMUR', 'G22SENRSMI', 'G22SENOWRI',  'DEM_PRIM_VOTE22', 'OTH_PRIM_VOTE22', 'REP_PRIM_VOTE22', 'DEM_PRIM_REG22', 'NON_PRIM_REG22', 'REP_PRIM_REG22', 'VOTED_GEN22',
                               'TOTAL_POP','TOTAL_VAP','WHT_NHSP_VAP', 'AIAN_NHSP_VAP', 'ASN_NHSP_VAP', 'NHOPI_NHSP_VAP', 'OTH_NHSP_VAP', 'OTH_HISP_VAP', 'HISP_VAP', 'ANY_PART_BLK_NHSP_VAP', 'ANY_PART_BLK_VAP', 
                               'CVAP_TOT21','CVAP_WHT21', 'CVAP_BLK21', 'CVAP_AIA21', 'CVAP_ASN21', 'CVAP_NHP21', 'CVAP_2OM21', 'CVAP_HSP21']

joined_demo_votes = joined_demo_votes[joined_demo_votes_col_order]

Extract data as a CSV

In [28]:
joined_demo_votes.to_csv('./wa_rpv_2020_precincts.csv',index=False)